In [15]:
import pandas as pd
import os

def filter_and_clean_excel(input_path, output_path, columns_to_keep, sheet_name=0, diff_columns=None):
    """
    טוען קובץ Excel (xlsx), בוחר עמודות, מוחק שורות חסרות,
    מחשב עמודות הפרש (אם סופקו), ושומר כ-CSV.

    Args:
        input_path (str): נתיב לקובץ ה-Excel.
        output_path (str): נתיב לשמירת קובץ ה-CSV החדש.
        columns_to_keep (list): רשימת העמודות לשמור (חייבת לכלול עמודות לחישוב).
        sheet_name (str or int): שם הגיליון (ברירת מחדל: 0).
        diff_columns (dict): מילון להגדרת עמודות הפרש.
                           פורמט: {'שם_עמודה_חדשה': ('עמודה_A', 'עמודה_B')}
                           החישוב יהיה: עמודה_A - עמודה_B
    """
    print(f"מתחיל בתהליך...")

    try:
        # 1. טעינת הנתונים מקובץ אקסל
        print(f"טוען נתונים מקובץ Excel: {input_path} (גיליון: {sheet_name})")
        df = pd.read_excel(
            input_path,
            sheet_name=sheet_name,
            usecols=columns_to_keep
        )
        original_rows = len(df)
        print(f"נתונים נטענו. צורה מקורית (רק עמודות נבחרות): {df.shape}")

    except FileNotFoundError:
        print(f"שגיאה: הקובץ לא נמצא בנתיב: {input_path}")
        return
    except ValueError as e:
        print(f"שגיאה: ייתכן שאחת העמודות שביקשת אינה קיימת. {e}")
        return
    except Exception as e:
        print(f"אירעה שגיאה בטעינת הקובץ: {e}")
        print("הערה: ודא שהספרייה 'openpyxl' מותקנת ('pip install openpyxl')")
        return

    # 2. מחיקת שורות עם ערכים חסרים (NaN)
    print("מוחק שורות עם ערכים חסרים...")
    df_cleaned = df.dropna()
    cleaned_rows = len(df_cleaned)

    rows_dropped = original_rows - cleaned_rows
    print(f"נמחקו {rows_dropped} שורות.")

    # 3. הוספת עמודות הפרש (החלק החדש)
    if diff_columns:
        print("מחשב עמודות הפרש...")
        # חשוב: השתמש ב-copy() כדי למנוע אזהרת SettingWithCopyWarning
        df_cleaned = df_cleaned.copy()

        for new_col, (col_a, col_b) in diff_columns.items():
            if col_a in df_cleaned.columns and col_b in df_cleaned.columns:

                # ודא שהעמודות הן מספריות לפני החיסור
                # errors='coerce' יהפוך טקסט (כמו "N/A") ל-NaN
                col_a_numeric = pd.to_numeric(df_cleaned[col_a], errors='coerce')
                col_b_numeric = pd.to_numeric(df_cleaned[col_b], errors='coerce')

                # בצע את החישוב
                df_cleaned[new_col] = col_a_numeric - col_b_numeric
                print(f"  נוצרה עמודה: {new_col} = {col_a} - {col_b}")
            else:
                print(f"  אזהרה: לא ניתן ליצור '{new_col}'. אחת מהעמודות ({col_a}, {col_b}) חסרה.")

        # 4. ניקוי נוסף
        # אם החישוב או ההמרה ל-numeric יצרו ערכי NaN חדשים, ננקה גם אותם
        # (זה שומר על הכלל שלך "למחוק שורות עם ערכים ריקים")
        final_cols_to_check = list(diff_columns.keys())
        df_cleaned = df_cleaned.dropna(subset=final_cols_to_check)
        final_rows_dropped = cleaned_rows - len(df_cleaned)

        if final_rows_dropped > 0:
            print(f"נמחקו {final_rows_dropped} שורות נוספות עקב חישוב הפרש (למשל, טקסט בעמודה).")

    print(f"צורה סופית: {df_cleaned.shape}")

    # 5. שמירת הקובץ החדש כ-CSV
    try:
        output_dir = os.path.dirname(output_path)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)

        df_cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"\n✅ הקובץ הנקי נשמר בהצלחה ב: {output_path}")

    except Exception as e:
        print(f"אירעה שגיאה בשמירת הקובץ: {e}")

# ===================================================================
#           דוגמת שימוש
# ===================================================================

# 1. 💡 הגדר את *כל* העמודות שאתה צריך (גם לחישוב וגם לשמירה)
my_selected_columns = [
    'Subject_Code',
    'b_DES_average',
    'after_DES_total',
    'b_DERS_total',
    'after_DERS_total',
     "after_DERS_Nonacceptance_Emotional_Responses",
     "after_DERS_Goal_Directed_Behavior",
     "after_DERS_Impulse_Control",
     "after_DERS_Lack_Emotional_Awareness",
     "after_DERS_Emotion_Regulation_Strategies",
     "after_DERS_Lack_Emotional_Clarity" ,
]

# 2. 💡 הגדר את חישובי ההפרש
#    פורמט: 'שם_חדש': ('ממנו_לחסר', 'מה_לחסר')
my_diff_calculations = {
    'diff_DES': ('after_DES_total', 'b_DES_average'),
    'diff_DERS': ('after_DERS_total', 'b_DERS_total')
}

# 3. 💡 הגדר את נתיבי הקבצים
input_file_path = 'data/q_data/Study_Questionnaire_Responses_October.xlsx'
output_file_path = 'data/only_Q_outputs/combined/regression/regression_parameters.csv'

# 4. קריאה לפונקציה
filter_and_clean_excel(
    input_path=input_file_path,
    output_path=output_file_path,
    columns_to_keep=my_selected_columns,
    diff_columns=my_diff_calculations
)

מתחיל בתהליך...
טוען נתונים מקובץ Excel: data/q_data/Study_Questionnaire_Responses_October.xlsx (גיליון: 0)
נתונים נטענו. צורה מקורית (רק עמודות נבחרות): (208, 11)
מוחק שורות עם ערכים חסרים...
נמחקו 71 שורות.
מחשב עמודות הפרש...
  נוצרה עמודה: diff_DES = after_DES_total - b_DES_average
  נוצרה עמודה: diff_DERS = after_DERS_total - b_DERS_total
צורה סופית: (137, 13)

✅ הקובץ הנקי נשמר בהצלחה ב: data/only_Q_outputs/combined/regression/regression_parameters.csv


In [22]:
import os
import pandas as pd

# ---------- 1. Paths ----------
t1_file = "data/T1_first_session_data.xlsx"  # the multi-sheet T1 file
regression_file = "data/only_Q_outputs/combined/regression/regression_parameters.csv"
pca_file = "data/only_Q_outputs/combined/combined_pca_components.csv"

# Folder where you want the new CSVs:
output_dir = "data/only_Q_outputs/combined/regression"
os.makedirs(output_dir, exist_ok=True)

# ---------- 2. Define the sheets you want to create CSVs for ----------
sheets = [
    "global_parameters",
    "subcortical_vol",
    "cortical_vol_lr",
    "cortical_lobes_lr",
    "cortical_lobes_full_brain",
    "cortical_networks_full_brain",
]

# ---------- 3. Load regression + PCA data ----------
reg_df = pd.read_csv(regression_file)
pca_df = pd.read_csv(pca_file)

# Normalize column names
reg_df.columns = reg_df.columns.str.lower()
pca_df.columns = pca_df.columns.str.lower()

# Ensure we have subject_code and time_point
# (rename here if your columns are a bit different)
# reg_df = reg_df.rename(columns={"your_reg_col": "subject_code"})
# pca_df = pca_df.rename(columns={"your_pca_col": "subject_code"})

# ---------- 4. Keep only time point "b" ----------
# If time_point is exactly "b":
mask_b_pca = pca_df["timepoint"].astype(str).str.contains("b", case=False, na=False)

reg_b = reg_df.copy()
pca_b = pca_df[mask_b_pca].copy()

# ---------- 5. Merge regression + PCA once (for all subjects) ----------
# Merge on both subject_code and time_point to be safe
merged_all = reg_b.merge(
    pca_b,
    on=["subject_code"],
    how="inner",
    suffixes=("_reg", "_pc")
)

# Optionally drop rows that still have NaNs
merged_all = merged_all.dropna(how="any")

print(f"Total rows after merging regression + PCs (time point b): {len(merged_all)}")

# ---------- 6. For each T1 sheet, subset by subjects and save a CSV ----------
for sheet_name in sheets:
    print(f"Processing sheet: {sheet_name}")

    # Load that sheet (just to get the subject list for this modality)
    t1_df = pd.read_excel(t1_file, sheet_name=sheet_name)
    t1_df.columns = t1_df.columns.str.lower()

    if "subject_code" not in t1_df.columns:
        raise ValueError(f"'subject_code' column not found in sheet '{sheet_name}'")

    # Subjects present in this sheet
    sheet_subjects = t1_df["subject_code"].unique()

    # Subset merged regression+PCs to these subjects
    sheet_merged = merged_all[merged_all["subject_code"].isin(sheet_subjects)].copy()

    # If you also want to remove any remaining NaNs (just in case):
    sheet_merged = sheet_merged.dropna(how="any")

    # ---------- 7. Save per-sheet CSV ----------
    output_file = os.path.join(
        output_dir,
        f"{sheet_name}_regression_pcs_b.csv"
    )
    sheet_merged.to_csv(output_file, index=False)

    print(f"  Saved {len(sheet_merged)} rows to: {output_file}")

print("Done! All regression + PC CSVs (time point b) saved in the 'regression' folder.")


Total rows after merging regression + PCs (time point b): 114
Processing sheet: global_parameters
  Saved 80 rows to: data/only_Q_outputs/combined/regression\global_parameters_regression_pcs_b.csv
Processing sheet: subcortical_vol
  Saved 80 rows to: data/only_Q_outputs/combined/regression\subcortical_vol_regression_pcs_b.csv
Processing sheet: cortical_vol_lr
  Saved 80 rows to: data/only_Q_outputs/combined/regression\cortical_vol_lr_regression_pcs_b.csv
Processing sheet: cortical_lobes_lr
  Saved 80 rows to: data/only_Q_outputs/combined/regression\cortical_lobes_lr_regression_pcs_b.csv
Processing sheet: cortical_lobes_full_brain
  Saved 80 rows to: data/only_Q_outputs/combined/regression\cortical_lobes_full_brain_regression_pcs_b.csv
Processing sheet: cortical_networks_full_brain
  Saved 80 rows to: data/only_Q_outputs/combined/regression\cortical_networks_full_brain_regression_pcs_b.csv
Done! All regression + PC CSVs (time point b) saved in the 'regression' folder.


In [2]:
import pandas as pd

# ---------- 1. Paths ----------
t1_file = "data/T1_first_session_data.xlsx"      # this is the file you already uploaded
subjects_file = "data/only_Q_outputs/combined/regression_parameters.csv"  # <-- change to your second file name

# ---------- 2. Load the sheet you want from the T1 file ----------
# Choose one of:
#   "global_parameters"
#   "subcortical_vol"
#   "cortical_vol_lr"
#   "cortical_lobes_lr"
#   "cortical_lobes_full_brain"
#   "cortical_networks_full_brain"

sheet_name = "cortical_networks_full_brain"  # <-- change to the sheet you want

t1_df = pd.read_excel(t1_file, sheet_name=sheet_name)

# ---------- 3. Load the subject_code file ----------
# If it’s an Excel:
subjects_df = pd.read_csv(subjects_file)
# If it’s CSV instead, use:
# subjects_df = pd.read_csv(subjects_file)

# ---------- 4. Normalize column names (to avoid Subject_Code vs subject_code issues) ----------
t1_df.columns = t1_df.columns.str.lower()
subjects_df.columns = subjects_df.columns.str.lower()

# Make sure both have a 'subject_code' column
# (rename here if your column is called differently)
# subjects_df = subjects_df.rename(columns={"your_col_name": "subject_code"})

# ---------- 5. Merge on subject_code ----------
# subjects_df is treated as the main list of subjects
merged = subjects_df.merge(t1_df, on="subject_code", how="left")

# ---------- 6. Drop rows with ANY NaNs ----------
# This removes any line that contains at least one NaN value
merged_no_nan = merged.dropna(how="any")

# ---------- 7. Save result ----------
output_file = "data/merged_no_nan.csv"
merged_no_nan.to_csv(output_file, index=False)

print(f"Done! Saved merged file without NaNs to: {output_file}")


Done! Saved merged file without NaNs to: data/merged_no_nan.csv


In [ ]:
import os
import pandas as pd

# ---------- 1. Paths ----------
t1_file = "data/T1_first_session_data.xlsx"  # the multi-sheet T1 file
regression_file = "data/only_Q_outputs/combined/regression_parameters.csv"
pca_file = "data/combined_pca_components.csv"

# Folder where you want the new CSVs:
output_dir = "data/only_Q_outputs/combined/regression"
os.makedirs(output_dir, exist_ok=True)

# ---------- 2. Define the sheets you want to create CSVs for ----------
sheets = [
    "global_parameters",
    "subcortical_vol",
    "cortical_vol_lr",
    "cortical_lobes_lr",
    "cortical_lobes_full_brain",
    "cortical_networks_full_brain",
]

# ---------- 3. Load regression + PCA data ----------
reg_df = pd.read_csv(regression_file)
pca_df = pd.read_csv(pca_file)

# Normalize column names
reg_df.columns = reg_df.columns.str.lower()
pca_df.columns = pca_df.columns.str.lower()

# Ensure we have subject_code and time_point
# (rename here if your columns are a bit different)
# reg_df = reg_df.rename(columns={"your_reg_col": "subject_code"})
# pca_df = pca_df.rename(columns={"your_pca_col": "subject_code"})

# ---------- 4. Keep only time point "b" ----------
# If time_point is exactly "b":
mask_b_reg = reg_df["time_point"].astype(str).str.contains("b", case=False, na=False)
mask_b_pca = pca_df["time_point"].astype(str).str.contains("b", case=False, na=False)

reg_b = reg_df[mask_b_reg].copy()
pca_b = pca_df[mask_b_pca].copy()

# ---------- 5. Merge regression + PCA once (for all subjects) ----------
# Merge on both subject_code and time_point to be safe
merged_all = reg_b.merge(
    pca_b,
    on=["subject_code", "time_point"],
    how="inner",
    suffixes=("_reg", "_pc")
)

# Optionally drop rows that still have NaNs
merged_all = merged_all.dropna(how="any")

print(f"Total rows after merging regression + PCs (time point b): {len(merged_all)}")

# ---------- 6. For each T1 sheet, subset by subjects and save a CSV ----------
for sheet_name in sheets:
    print(f"Processing sheet: {sheet_name}")

    # Load that sheet (just to get the subject list for this modality)
    t1_df = pd.read_excel(t1_file, sheet_name=sheet_name)
    t1_df.columns = t1_df.columns.str.lower()

    if "subject_code" not in t1_df.columns:
        raise ValueError(f"'subject_code' column not found in sheet '{sheet_name}'")

    # Subjects present in this sheet
    sheet_subjects = t1_df["subject_code"].unique()

    # Subset merged regression+PCs to these subjects
    sheet_merged = merged_all[merged_all["subject_code"].isin(sheet_subjects)].copy()

    # If you also want to remove any remaining NaNs (just in case):
    sheet_merged = sheet_merged.dropna(how="any")

    # ---------- 7. Save per-sheet CSV ----------
    output_file = os.path.join(
        output_dir,
        f"{sheet_name}_regression_pcs_b.csv"
    )
    sheet_merged.to_csv(output_file, index=False)

    print(f"  Saved {len(sheet_merged)} rows to: {output_file}")

print("Done! All regression + PC CSVs (time point b) saved in the 'regression' folder.")


## pc1 predictio


In [4]:
# import pandas as pd
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import KFold, cross_validate
# from sklearn.metrics import mean_squared_error, r2_score
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats # משמש לחישוב P-value
#
# # ===================================================================
# #           חלק 1: הגדרות נתיבים ופרמטרים
# # ===================================================================
#
# # נתיבי הקבצים - חובה לעדכן את נתיב נתוני ה-PC!
# # קובץ זה הוא בפורמט LONG: (Subject_Code | timepoint | PC1 | PC2 |...)
# PC_DATA_LONG_PATH = "data/only_Q_outputs/combined/combined_pca_components.csv"
# raw_data_path = "data/only_Q_outputs/combined/regression_parameters.csv"
#
# # תיקייה לשמירת כל הגרפים
# PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/regression_PC1_SIMPLE_plots_PIVOTED"
# # קובץ לשמירת טבלת הסיכום
# SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/regression_PC1_SIMPLE_summary_table_PIVOTED.csv"
#
# # 💡💡💡
# # --- הגדר כאן את הפרמטרים ---
# # 💡💡💡
#
# # 1. רשימת המשתנים *הקבועים* שאתה רוצה לנבא (Y)
# Y_VARIABLES_TO_PREDICT = [
#     'b_DES_average',
#     'after_DES_total',
#     "b_DERS_total",
#     "after_DERS_total",
#     "diff_DERS",
#     "diff_DES"
# ]
#
# # 2. רשימת נקודות הזמן וה-PC's לשימוש
# PC_TIMEPOINTS = ['b', 't1', 't2', 't3', 'after']
# PC_COMPONENT = 'PC1' # *** משתנה מנבא יחיד: PC1 ***
#
# # 3. הגדרות לאימות צולב
# N_SPLITS = 5
# RANDOM_STATE = 42
#
# df_for_regression = None
#
# try:
#     # ===================================================================
#     #           חלק 1: הכנת הנתונים (טעינה, Pivot ומיזוג)
#     # ===================================================================
#
#     # --- 1א: טעינת נתוני ה-PC ב-LONG FORMAT ---
#     print(f"Loading PC data in Long Format from {PC_DATA_LONG_PATH}...")
#     pc_df_long = pd.read_csv(PC_DATA_LONG_PATH)
#
#     # --- 1ב: יצירת WIDE FORMAT (Pivot) רק עבור PC1 ---
#     print(f"Pivoting to Wide Format using '{PC_COMPONENT}' values...")
#
#     # ודא שכל המידע הדרוש קיים לפני הפיווט
#     if 'Subject_Code' not in pc_df_long.columns or 'timepoint' not in pc_df_long.columns or PC_COMPONENT not in pc_df_long.columns:
#         raise ValueError(f"Missing essential columns (Subject_Code, timepoint, or {PC_COMPONENT}) in the PC data.")
#
#     pc_df_wide = pc_df_long.pivot(
#         index='Subject_Code',
#         columns='timepoint',
#         values=PC_COMPONENT # משתמשים רק ב-PC1
#     ).reset_index()
#
#     # המרה ל-float64
#     for tp in PC_TIMEPOINTS:
#         if tp in pc_df_wide.columns:
#             pc_df_wide[tp] = pc_df_wide[tp].astype(np.float64, errors='ignore')
#
#     # --- 1ג: הכנת נתוני ה-Y ומיזוג ---
#     print("Loading Y variables and merging...")
#     columns_to_load = ['Subject_Code'] + Y_VARIABLES_TO_PREDICT
#     raw_df = pd.read_csv(raw_data_path, usecols=lambda c: c in columns_to_load)
#
#     # המרה ל-float64
#     for y_col in Y_VARIABLES_TO_PREDICT:
#         if y_col in raw_df.columns:
#             raw_df[y_col] = raw_df[y_col].astype(np.float64, errors='ignore')
#
#     df_for_regression = pd.merge(
#         pc_df_wide,
#         raw_df,
#         on='Subject_Code',
#         how='inner'
#     )
#
#     print("--- Data successfully merged (Wide Format for PC1) ---")
#     print(df_for_regression.head())
#
#     # יצירת תיקיית הפלט לגרפים
#     os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)
#
# except FileNotFoundError:
#     print(f"שגיאה: אחד מקובצי הנתונים לא נמצא. ודא נתיבים.")
#     df_for_regression = None
# except Exception as e:
#     # הצגת השגיאה שהתרחשה (כדי לדעת איפה הבעיה)
#     print(f"אירעה שגיאה: {e}")
#     df_for_regression = None
#
# # ===================================================================
# #           חלק 2: הרצת הניתוח בלולאה (רגרסיה פשוטה ו-CV)
# # ===================================================================
#
# all_results_list = []
#
# if df_for_regression is not None:
#
#     # --- התחלת הלולאות ---
#     for y_col in Y_VARIABLES_TO_PREDICT:
#         print(f"\n=======================================================")
#         print(f"  Processing Y-Variable: {y_col}")
#         print(f"=======================================================")
#
#         # *** הלולאה רצה על כל נקודת זמן כעמודת מנבא ***
#         for predictor_col_name in PC_TIMEPOINTS:
#
#             if predictor_col_name not in df_for_regression.columns:
#                 print(f"--- Skipping: Timepoint column '{predictor_col_name}' not found after pivot.")
#                 continue
#
#             timepoint_name = predictor_col_name # שם העמודה (לדוגמה 'b') הוא שם נקודת הזמן
#
#             print(f"\n--- Running Simple Regression: '{y_col}' ~ '{predictor_col_name}' ({PC_COMPONENT} at {timepoint_name}) ---")
#
#             # 4. הכנת X ו-Y, וניקוי ערכים חסרים
#             data_clean = df_for_regression[[predictor_col_name, y_col]].dropna()
#
#             if len(data_clean) < N_SPLITS * 2:
#                 print(f"  Skipping: Not enough data (N={len(data_clean)})")
#                 continue
#
#             # X חייב להיות מטריצה דו-ממדית עבור sklearn
#             X = data_clean[[predictor_col_name]]
#             y = data_clean[y_col]
#
#             # --- 💡 חישוב P-value (P-value עבור מקדם בטא של הרגרסיה)
#             try:
#                 slope, intercept, r_value, p_value, std_err = stats.linregress(X[predictor_col_name], y)
#                 t_stat = slope / std_err
#             except ValueError:
#                 t_stat, p_value, slope, intercept = np.nan, np.nan, np.nan, np.nan
#             # -------------------------------------------------------------
#
#             # 5. שלב א': הערכת המודל (אימות צולב)
#             model_cv = LinearRegression()
#             kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
#             scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}
#
#             # *** התיקון כאן: cv=kfold ***
#             cv_scores = cross_validate(model_cv, X, y, cv=kfold, scoring=scoring)
#
#             mean_r2 = np.mean(cv_scores['test_r_squared'])
#             std_r2 = np.std(cv_scores['test_r_squared'])
#             mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))
#             print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")
#
#             # 6. שלב ב': מקדמים סופיים
#             alpha = intercept
#             beta = slope
#             print(f"  Intercept (Alpha): {alpha:.4f} | Slope (Beta): {beta:.4f}")
#             print(f"  P-value (Beta): {p_value:.4f}")
#
#             # ===============================================================
#             #               💡💡💡 תנאי: הצגה רק אם R² חיובי 💡💡💡
#             # ===============================================================
#             if mean_r2 > 0:
#                 print(f"  ✅ R² is positive ({mean_r2:.3f}). Plotting graph.")
#
#                 # 7. שלב ג': שרטוט הגרף (גרף פיזור)
#                 plt.figure(figsize=(8, 6))
#
#                 # גרף פיזור של הנתונים
#                 sns.scatterplot(x=predictor_col_name, y=y_col, data=data_clean, color='darkblue', alpha=0.7)
#
#                 # הוספת קו הרגרסיה
#                 sns.regplot(x=predictor_col_name, y=y_col, data=data_clean, scatter=False, color='red', line_kws={'lw': 2, 'linestyle': '-'})
#
#                 plt.title(f'Prediction: {y_col} by {PC_COMPONENT} at Timepoint {timepoint_name}')
#                 plt.xlabel(f'{PC_COMPONENT} ({timepoint_name})')
#                 plt.ylabel(f'{y_col}')
#
#                 plt.text(0.05, 0.95,
#                          f'$R^2$ (CV) = {mean_r2:.3f}\nSlope (Beta) = {beta:.3f}\n$P$ (Beta) = {p_value:.4f}',
#                          transform=plt.gca().transAxes,
#                          verticalalignment='top',
#                          bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.7))
#
#                 plt.tight_layout()
#
#                 # שמירת הגרף עם שם ייחודי
#                 plot_save_path = os.path.join(PLOTS_SAVE_DIR, f"simple_reg_{y_col}_vs_{PC_COMPONENT}_{predictor_col_name}.png")
#                 plt.savefig(plot_save_path, dpi=100)
#                 plt.close()
#                 print(f"  Plot saved to {plot_save_path}")
#             else:
#                 print(f"  ❌ R² is not positive ({mean_r2:.3f}). Skipping graph plotting.")
#             # ===============================================================
#
#             # 8. שמירת התוצאות לרשימה
#             all_results_list.append({
#                 'Y_Variable': y_col,
#                 'X_Variable (PC1_Timepoint)': predictor_col_name,
#                 'PC_Component': PC_COMPONENT,
#                 'Timepoint': timepoint_name,
#                 'R_Squared_CV': mean_r2,
#                 'R_Squared_Std': std_r2,
#                 'RMSE_CV': mean_rmse,
#                 'Alpha_Intercept': alpha,
#                 'Beta_Slope': beta,
#                 'T_Stat': t_stat,
#                 'P_Value': p_value,
#                 'N': len(y),
#                 'Plotted': mean_r2 > 0
#             })
#
#     print("\n\n--- כל הניתוחים הסתיימו ---")
#
#     # ===================================================================
#     #           חלק 3: יצירת טבלת סיכום
#     # ===================================================================
#
#     if all_results_list:
#         summary_df = pd.DataFrame(all_results_list)
#         # נמיין לפי R^2 יורד
#         summary_df = summary_df.sort_values(by=['Y_Variable', 'R_Squared_CV'], ascending=[True, False])
#
#         print("\n--- טבלת סיכום סופית (רגרסיה פשוטה PC1) ---")
#
#         summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
#         print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
#     else:
#         print("\n--- לא נוצרו תוצאות ---")
#
# else:
#     print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

Loading PC data in Long Format from data/only_Q_outputs/combined/combined_pca_components.csv...
שגיאה: אחד מקובצי הנתונים לא נמצא. ודא נתיבים.
הניתוח לא רץ כי הנתונים לא נטענו כראוי.


## clusters prediction


In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats # לספירת P-value (מובהקות)

# ===================================================================
#           חלק 1: הכנת הנתונים (טעינה, ומיזוג)
# ===================================================================

# נתיבי הקבצים - חובה לעדכן את נתיב האשכולות!
# קובץ זה צריך להיות בפורמט: (Subject_Code | b | t1 | t2 | t3 | after)
CLUSTER_DATA_WIDE_PATH = "data/only_Q_outputs/combined/timepoints_file_inverted_2.csv"
raw_data_path = "data/only_Q_outputs/combined/regression_parameters.csv"

# תיקייה לשמירת כל הגרפים - שינוי שם התיקייה כדי לשקף סינון
PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/cluster_ders_des_regression_CLUSTER_POSITIVE_R2_plots"
# קובץ לשמירת טבלת הסיכום
SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/clusters_regression_CLUSTER_WIDE_summary_table_filtered.csv"

# 💡💡💡
# --- הגדר כאן את הפרמטרים ---
# 💡💡💡

# 1. רשימת המשתנים *הקבועים* שאתה רוצה לנבא (Y) - (כמו קודם)
Y_VARIABLES_TO_PREDICT = [
    'b_DES_average',
    'after_DES_total',
    "b_DERS_total",
    "after_DERS_total",
    "diff_DERS",
    "diff_DES"
    # 💡 הוסף כאן עוד שמות עמודות
]

# 2. רשימת נקודות הזמן של האשכול (X)
# הרשימה הזו חייבת להתאים לשמות העמודות בקובץ האשכולות (CLUSTER_DATA_WIDE_PATH)
CLUSTER_TIMEPOINTS = ['before', 't1', 't2', 't3', 'after']

df_for_regression = None

try:
    # --- 1א: הכנת נתוני ה-X (Cluster) ---
    print("Loading Cluster data in Wide Format...")
    cluster_df_wide = pd.read_csv(CLUSTER_DATA_WIDE_PATH)

    # --- 1ב: הכנת נתוני ה-Y (כמו קודם) ---
    print("Loading Y variables...")
    columns_to_load = ['Subject_Code'] + Y_VARIABLES_TO_PREDICT
    raw_df = pd.read_csv(raw_data_path, usecols=lambda c: c in columns_to_load)

    # --- 1ג: מיזוג ---
    df_for_regression = pd.merge(
        cluster_df_wide,
        raw_df,
        on='Subject_Code'
    )

    # ודא שכל עמודות הקלאסטר (הן ה-X שלנו) הן מספרים שלמים (0 או 1)
    for tp in CLUSTER_TIMEPOINTS:
         df_for_regression[tp] = df_for_regression[tp].astype('Int64', errors='ignore')

    print("--- Data successfully merged ---")
    print(df_for_regression.head())

    # יצירת תיקיית הפלט לגרפים
    os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)

except FileNotFoundError:
    print("שגיאה: אחד מקובצי הנתונים לא נמצא. ודא ש-CLUSTER_DATA_WIDE_PATH ו-raw_data_path נכונים.")
    df_for_regression = None
except ValueError as e:
    print(f"שגיאה בטעינת הנתונים. ייתכן שאחת העמודות ב-Y_VARIABLES_TO_PREDICT אינה קיימת. {e}")
    df_for_regression = None
except Exception as e:
    print(f"אירעה שגיאה: {e}")
    df_for_regression = None

# ===================================================================
#           חלק 2: הרצת הניתוח בלולאה (רגרסיה ו-CV)
# ===================================================================

# הגדרות נוספות
N_SPLITS = 5
RANDOM_STATE = 42
all_results_list = []

if df_for_regression is not None:

    # --- התחלת הלולאות ---
    for y_col in Y_VARIABLES_TO_PREDICT:
        print(f"\n=======================================================")
        print(f"  Processing Y-Variable: {y_col}")
        print(f"=======================================================")

        for tp in CLUSTER_TIMEPOINTS:

            # 3. בניית שם המשתנה המנבא (X)
            X_COL_NAME = tp

            if X_COL_NAME not in df_for_regression.columns:
                print(f"--- Skipping: Cluster column '{X_COL_NAME}' not found in merged data.")
                continue

            print(f"\n--- Running: '{y_col}' ~ '{X_COL_NAME}' (Cluster) ---")

            # 4. הכנת X ו-Y, וניקוי ערכים חסרים
            data_clean = df_for_regression[[X_COL_NAME, y_col]].dropna()

            if len(data_clean) < N_SPLITS * 2:
                print(f"  Skipping: Not enough data (N={len(data_clean)})")
                continue

            X = data_clean[[X_COL_NAME]]
            y = data_clean[y_col]

            # --- 💡 חישוב P-value באמצעות סטטיסטיקת T (t-test)
            if X[X_COL_NAME].nunique() < 2:
                print(f"  Skipping: Only one cluster ({X[X_COL_NAME].iloc[0]}) in the data for this timepoint.")
                continue

            group0 = data_clean[data_clean[X_COL_NAME] == 0][y_col]
            group1 = data_clean[data_clean[X_COL_NAME] == 1][y_col]

            if len(group0) > 1 and len(group1) > 1:
                t_stat, p_value = stats.ttest_ind(group1, group0, equal_var=False, nan_policy='omit')
            else:
                t_stat, p_value = np.nan, np.nan
            # -------------------------------------------------------------

            # 5. שלב א': הערכת המודל (אימות צולב)
            model_cv = LinearRegression()
            kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
            scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}

            cv_scores = cross_validate(model_cv, X, y, cv=kfold, scoring=scoring)

            mean_r2 = np.mean(cv_scores['test_r_squared'])
            std_r2 = np.std(cv_scores['test_r_squared'])
            mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))
            print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")

            # 6. שלב ב': אימון מודל סופי (לקבלת מקדמים)
            model_final = LinearRegression()
            model_final.fit(X, y)
            alpha = model_final.intercept_
            beta = model_final.coef_[0]
            print(f"  Mean Cluster 0 (Alpha): {alpha:.4f} | Diff Clust 1-0 (Beta): {beta:.4f}")
            print(f"  P-value (t-test): {p_value:.4f}")

            # ===============================================================
            #               💡💡💡 תנאי חדש: הצגה רק אם R² חיובי 💡💡💡
            # ===============================================================
            if mean_r2 > 0:
                print(f"  ✅ R² is positive ({mean_r2:.3f}). Plotting graph.")

                # 7. שלב ג': שרטוט הגרף
                plt.figure(figsize=(8, 6))
                sns.boxplot(x=X_COL_NAME, y=y_col, data=data_clean, palette='pastel')
                sns.stripplot(x=X_COL_NAME, y=y_col, data=data_clean, color='black', size=4, alpha=0.7, jitter=True)

                # הוספת קו הרגרסיה (ממוצעים)
                plt.axhline(alpha, color='red', linestyle='--', label=f'Mean Clust 0 ({alpha:.2f})')
                plt.axhline(alpha + beta, color='blue', linestyle='--', label=f'Mean Clust 1 ({alpha + beta:.2f})')

                plt.title(f'Prediction: {y_col} by Cluster ({X_COL_NAME})')
                plt.xlabel(f'Cluster (0 vs 1) at Timepoint: {X_COL_NAME}')
                plt.ylabel(f'{y_col}')

                plt.text(0.05, 0.95,
                         f'$R^2$ (CV) = {mean_r2:.3f}\nDiff (Beta) = {beta:.3f}\n$P$ (t-test) = {p_value:.4f}',
                         transform=plt.gca().transAxes,
                         verticalalignment='top',
                         bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.7))

                plt.tight_layout()

                # שמירת הגרף עם שם ייחודי
                plot_save_path = os.path.join(PLOTS_SAVE_DIR, f"cluster_reg_{y_col}_vs_{X_COL_NAME}.png")
                plt.savefig(plot_save_path, dpi=100)
                plt.close()
                print(f"  Plot saved to {plot_save_path}")
            else:
                print(f"  ❌ R² is not positive ({mean_r2:.3f}). Skipping graph plotting.")
            # ===============================================================
            #                         סוף התנאי
            # ===============================================================

            # 8. שמירת התוצאות לרשימה (שומרים את כל התוצאות, גם שליליות, בטבלה)
            all_results_list.append({
                'Y_Variable': y_col,
                'X_Variable (Cluster Timepoint)': X_COL_NAME,
                'R_Squared_CV': mean_r2,
                'R_Squared_Std': std_r2,
                'RMSE_CV': mean_rmse,
                'Alpha_Mean_Clust0': alpha,
                'Beta_Diff_Clust1_Minus_Clust0': beta,
                'T_Stat': t_stat,
                'P_Value': p_value,
                'N': len(y),
                'Plotted': mean_r2 > 0 # הוספת אינדיקציה האם הגרף שורטט
            })

    print("\n\n--- כל הניתוחים הסתיימו ---")

    # ===================================================================
    #           חלק 3: יצירת טבלת סיכום
    # ===================================================================

    if all_results_list:
        summary_df = pd.DataFrame(all_results_list)
        # כדי להקל על הסינון, נמיין לפי R^2 יורד
        summary_df = summary_df.sort_values(by=['Y_Variable', 'R_Squared_CV'], ascending=[True, False])

        print("\n--- טבלת סיכום סופית ---")

        summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
        print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
    else:
        print("\n--- לא נוצרו תוצאות ---")

else:
    print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

Loading Cluster data in Wide Format...
שגיאה: אחד מקובצי הנתונים לא נמצא. ודא ש-CLUSTER_DATA_WIDE_PATH ו-raw_data_path נכונים.
הניתוח לא רץ כי הנתונים לא נטענו כראוי.


## only T1 predict clinical


In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats # משמש לחישוב P-value

# ===================================================================
#           חלק 1: הגדרות נתיבים ופרמטרים
# ===================================================================

# נתיב הקובץ - הקובץ מכיל את כל הנתונים הנדרשים (X ו-Y)
# 💡 ודא שהנתיב הזה נכון לקובץ הממוזג שלך.
DATA_PATH = "data/merged_no_nan.csv"

# תיקייה לשמירת כל הגרפים
PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/SIMPLE_REGRESSION_CUSTOM_PREDICTORS_plots"
# קובץ לשמירת טבלת הסיכום
SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/T1_REGRESSION_CUSTOM_PREDICTORS_summary_table.csv"

# 💡💡💡
# --- הגדר כאן את הפרמטרים ---
# 💡💡💡
print(pd.read_csv(DATA_PATH).columns.tolist())
# 1. רשימת המשתנים *הקבועים* שאתה רוצה לנבא (Y)
Y_VARIABLES_TO_PREDICT = [
    'b_des_average',
    'after_des_total',
    "b_ders_total",
    "after_ders_total",
    "diff_ders",
    "diff_des"
]

# 2. רשימת המשתנים המנבאים (X) שאתה רוצה להשתמש בהם
X_PREDICTORS = [
    'dmn_gm',
    'fpn_gm',
    'lim_gm',
    'smn_gm',
    'van_gm',
    'vis_gm'
]

# 3. הגדרות לאימות צולב
N_SPLITS = 5
RANDOM_STATE = 42

df_for_regression = None

try:
    # ===================================================================
    #           חלק 1: הכנת הנתונים (טעינה וניקוי)
    # ===================================================================

    print(f"Loading merged data (X and Y variables) from {DATA_PATH}...")

    # טוענים את כל העמודות הדרושות: Subject_Code, כל המנבאים (X) וכל המנובאים (Y)
    columns_to_load = ['Subject_Code'] + Y_VARIABLES_TO_PREDICT + X_PREDICTORS

    # טעינת הנתונים. אם הקובץ מכיל רק את העמודות הללו, אפשר להשתמש ב-usecols כדי להיות יעילים.
    df_for_regression = pd.read_csv(DATA_PATH, usecols=lambda c: c in columns_to_load)

    # המרה ל-float64 לכל העמודות הרלוונטיות
    all_var_cols = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    for col in all_var_cols:
        if col in df_for_regression.columns:
            # המרה יזומה ל-float64 תוך התעלמות מטעויות אם יש ערכים שאינם מספריים
            df_for_regression[col] = pd.to_numeric(df_for_regression[col], errors='coerce').astype(np.float64)

    print("--- Data successfully loaded (ready for regression) ---")
    print(df_for_regression.head())

    # יצירת תיקיית הפלט לגרפים
    os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)

except FileNotFoundError:
    print(f"שגיאה: קובץ הנתונים לא נמצא. ודא נתיב: {DATA_PATH}")
    df_for_regression = None
except Exception as e:
    print(f"אירעה שגיאה: {e}")
    df_for_regression = None

# -------------------------------------------------------------------
# הערה: מכאן והלאה, הקוד נשאר זהה לקוד הקודם, כיוון שהוא כבר מבצע
# את ניתוח הרגרסיה הפשוטה באופן נכון על ה-DataFrame הטעון.
# -------------------------------------------------------------------

# ===================================================================
#           חלק 2: הרצת הניתוח בלולאה (רגרסיה פשוטה ו-CV)
# ===================================================================

all_results_list = []

if df_for_regression is not None:

    # --- התחלת הלולאות ---
    for y_col in Y_VARIABLES_TO_PREDICT:
        print(f"\n=======================================================")
        print(f"  Processing Y-Variable: {y_col}")
        print(f"=======================================================")

        # *** הלולאה רצה על כל משתנה מנבא (X) מותאם אישית ***
        for predictor_col_name in X_PREDICTORS:

            if predictor_col_name not in df_for_regression.columns:
                print(f"--- Skipping: Predictor column '{predictor_col_name}' not found in data.")
                continue

            print(f"\n--- Running Simple Regression: '{y_col}' ~ '{predictor_col_name}' ---")

            # 4. הכנת X ו-Y, וניקוי ערכים חסרים (NaN)
            data_clean = df_for_regression[[predictor_col_name, y_col]].dropna()

            if len(data_clean) < N_SPLITS * 2:
                print(f"  Skipping: Not enough data (N={len(data_clean)}). Need at least {N_SPLITS*2} for CV.")
                continue

            # X חייב להיות מטריצה דו-ממדית עבור sklearn
            X = data_clean[[predictor_col_name]]
            y = data_clean[y_col]

            # --- 💡 חישוב P-value וסטטיסטיקות רגרסיה קלאסיות
            try:
                # משתמשים בעמודה ה-1D של X עבור linregress
                slope, intercept, r_value, p_value, std_err = stats.linregress(X[predictor_col_name], y)
                t_stat = slope / std_err
            except ValueError:
                t_stat, p_value, slope, intercept = np.nan, np.nan, np.nan, np.nan
            # -------------------------------------------------------------

            # 5. שלב א': הערכת המודל (אימות צולב - Cross-Validation)
            model_cv = LinearRegression()
            kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
            scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}

            cv_scores = cross_validate(model_cv, X, y, cv=kfold, scoring=scoring)

            mean_r2 = np.mean(cv_scores['test_r_squared'])
            std_r2 = np.std(cv_scores['test_r_squared'])
            mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))
            print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")

            # 6. שלב ב': מקדמים סופיים (מתוך stats.linregress)
            alpha = intercept
            beta = slope
            print(f"  Intercept (Alpha): {alpha:.4f} | Slope (Beta): {beta:.4f}")
            print(f"  P-value (Beta): {p_value:.4f}")

            # ===============================================================
            #               💡💡💡 תנאי: הצגה רק אם R² חיובי 💡💡💡
            # ===============================================================
            if mean_r2 > 0:
                print(f"  ✅ R² is positive ({mean_r2:.3f}). Plotting graph.")

                # 7. שלב ג': שרטוט הגרף (גרף פיזור)
                plt.figure(figsize=(8, 6))

                # גרף פיזור של הנתונים
                sns.scatterplot(x=predictor_col_name, y=y_col, data=data_clean, color='darkblue', alpha=0.7)

                # הוספת קו הרגרסיה
                sns.regplot(x=predictor_col_name, y=y_col, data=data_clean, scatter=False, color='red', line_kws={'lw': 2, 'linestyle': '-'})

                plt.title(f'Prediction: {y_col} by {predictor_col_name}')
                plt.xlabel(f'{predictor_col_name}')
                plt.ylabel(f'{y_col}')

                plt.text(0.05, 0.95,
                         f'$R^2$ (CV) = {mean_r2:.3f}\nSlope (Beta) = {beta:.3f}\n$P$ (Beta) = {p_value:.4f}',
                         transform=plt.gca().transAxes,
                         verticalalignment='top',
                         bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.7))

                plt.tight_layout()

                # שמירת הגרף עם שם ייחודי
                plot_save_path = os.path.join(PLOTS_SAVE_DIR, f"simple_reg_{y_col}_vs_{predictor_col_name}.png")
                plt.savefig(plot_save_path, dpi=100)
                plt.close()
                print(f"  Plot saved to {plot_save_path}")
            else:
                print(f"  ❌ R² is not positive ({mean_r2:.3f}). Skipping graph plotting.")
            # ===============================================================

            # 8. שמירת התוצאות לרשימה
            all_results_list.append({
                'Y_Variable': y_col,
                'X_Variable': predictor_col_name,
                'R_Squared_CV': mean_r2,
                'R_Squared_Std': std_r2,
                'RMSE_CV': mean_rmse,
                'Alpha_Intercept': alpha,
                'Beta_Slope': beta,
                'T_Stat': t_stat,
                'P_Value': p_value,
                'N': len(y),
                'Plotted': mean_r2 > 0
            })

    print("\n\n--- כל הניתוחים הסתיימו ---")

    # ===================================================================
    #           חלק 3: יצירת טבלת סיכום
    # ===================================================================

    if all_results_list:
        summary_df = pd.DataFrame(all_results_list)
        # נמיין לפי R^2 יורד
        summary_df = summary_df.sort_values(by=['Y_Variable', 'R_Squared_CV'], ascending=[True, False])

        print("\n--- טבלת סיכום סופית (רגרסיה פשוטה מנבאים מותאמים אישית) ---")

        summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
        print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
    else:
        print("\n--- לא נוצרו תוצאות ---")

else:
    print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

['subject_code', 'b_ders_total', 'b_des_average', 'after_des_total', 'after_ders_total', 'after_ders_nonacceptance_emotional_responses', 'after_ders_goal_directed_behavior', 'after_ders_impulse_control', 'after_ders_lack_emotional_awareness', 'after_ders_emotion_regulation_strategies', 'after_ders_lack_emotional_clarity', 'diff_des', 'diff_ders', 'dmn_gm', 'fpn_gm', 'lim_gm', 'smn_gm', 'van_gm', 'vis_gm']
Loading merged data (X and Y variables) from data/merged_no_nan.csv...
--- Data successfully loaded (ready for regression) ---
   b_ders_total  b_des_average  after_des_total  after_ders_total   diff_des  \
0          68.0       2.142857         0.000000              91.0  -2.142857   
1          67.0       4.642857         7.500000              69.0   2.857143   
2          95.0       7.142857         5.000000              87.0  -2.142857   
3          70.0      23.928571        10.357143              72.0 -13.571429   
4          69.0       2.500000         1.785714              47.

## pc2 prediction


In [7]:
# import pandas as pd
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import KFold, cross_validate
# from sklearn.metrics import mean_squared_error, r2_score
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats # משמש לחישוב P-value
#
# # ===================================================================
# #           חלק 1: הגדרות נתיבים ופרמטרים
# # ===================================================================
#
# # נתיבי הקבצים - חובה לעדכן את נתיב נתוני ה-PC!
# # קובץ זה הוא בפורמט LONG: (Subject_Code | timepoint | PC1 | PC2 |...)
# PC_DATA_LONG_PATH = "data/only_Q_outputs/combined/combined_pca_components.csv"
# raw_data_path = "data/only_Q_outputs/combined/regression_parameters.csv"
#
# # תיקייה לשמירת כל הגרפים
# PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/regression_PC2_SIMPLE_plots_PIVOTED"
# # קובץ לשמירת טבלת הסיכום
# SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/regression_PC2_SIMPLE_summary_table_PIVOTED.csv"
#
# # 💡💡💡
# # --- הגדר כאן את הפרמטרים ---
# # 💡💡💡
#
# # 1. רשימת המשתנים *הקבועים* שאתה רוצה לנבא (Y)
# Y_VARIABLES_TO_PREDICT = [
#     'b_DES_average',
#     'after_DES_total',
#     "b_DERS_total",
#     "after_DERS_total",
#     "diff_DERS",
#     "diff_DES"
# ]
#
# # 2. רשימת נקודות הזמן וה-PC's לשימוש
# PC_TIMEPOINTS = ['b', 't1', 't2', 't3', 'after']
# PC_COMPONENT = 'PC2' # *** משתנה מנבא יחיד: PC1 ***
#
# # 3. הגדרות לאימות צולב
# N_SPLITS = 5
# RANDOM_STATE = 42
#
# df_for_regression = None
#
# try:
#     # ===================================================================
#     #           חלק 1: הכנת הנתונים (טעינה, Pivot ומיזוג)
#     # ===================================================================
#
#     # --- 1א: טעינת נתוני ה-PC ב-LONG FORMAT ---
#     print(f"Loading PC data in Long Format from {PC_DATA_LONG_PATH}...")
#     pc_df_long = pd.read_csv(PC_DATA_LONG_PATH)
#
#     # --- 1ב: יצירת WIDE FORMAT (Pivot) רק עבור PC1 ---
#     print(f"Pivoting to Wide Format using '{PC_COMPONENT}' values...")
#
#     # ודא שכל המידע הדרוש קיים לפני הפיווט
#     if 'Subject_Code' not in pc_df_long.columns or 'timepoint' not in pc_df_long.columns or PC_COMPONENT not in pc_df_long.columns:
#         raise ValueError(f"Missing essential columns (Subject_Code, timepoint, or {PC_COMPONENT}) in the PC data.")
#
#     pc_df_wide = pc_df_long.pivot(
#         index='Subject_Code',
#         columns='timepoint',
#         values=PC_COMPONENT # משתמשים רק ב-PC1
#     ).reset_index()
#
#     # ודא ששמות העמודות לאחר ה-Pivot הם כמו ב-PC_TIMEPOINTS (b, t1,...)
#     # אין צורך לשנות שמות מכיוון ש-Pivot עושה זאת אוטומטית אם יש רק עמודת ערך אחת
#
#     # המרה ל-float64
#     for tp in PC_TIMEPOINTS:
#         if tp in pc_df_wide.columns:
#             pc_df_wide[tp] = pc_df_wide[tp].astype(np.float64, errors='ignore')
#
#     # --- 1ג: הכנת נתוני ה-Y ומיזוג ---
#     print("Loading Y variables and merging...")
#     columns_to_load = ['Subject_Code'] + Y_VARIABLES_TO_PREDICT
#     raw_df = pd.read_csv(raw_data_path, usecols=lambda c: c in columns_to_load)
#
#     # המרה ל-float64
#     for y_col in Y_VARIABLES_TO_PREDICT:
#         if y_col in raw_df.columns:
#             raw_df[y_col] = raw_df[y_col].astype(np.float64, errors='ignore')
#
#     df_for_regression = pd.merge(
#         pc_df_wide,
#         raw_df,
#         on='Subject_Code',
#         how='inner'
#     )
#
#     print("--- Data successfully merged (Wide Format for PC1) ---")
#     print(df_for_regression.head())
#
#     # יצירת תיקיית הפלט לגרפים
#     os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)
#
# except FileNotFoundError:
#     print(f"שגיאה: אחד מקובצי הנתונים לא נמצא. ודא נתיבים.")
#     df_for_regression = None
# except Exception as e:
#     # הצגת השגיאה שהתרחשה (כדי לדעת איפה הבעיה)
#     print(f"אירעה שגיאה: {e}")
#     df_for_regression = None
#
# # ===================================================================
# #           חלק 2: הרצת הניתוח בלולאה (רגרסיה פשוטה ו-CV)
# # ===================================================================
#
# all_results_list = []
#
# if df_for_regression is not None:
#
#     # --- התחלת הלולאות ---
#     for y_col in Y_VARIABLES_TO_PREDICT:
#         print(f"\n=======================================================")
#         print(f"  Processing Y-Variable: {y_col}")
#         print(f"=======================================================")
#
#         # *** הלולאה רצה על כל נקודת זמן כעמודת מנבא ***
#         for predictor_col_name in PC_TIMEPOINTS:
#
#             if predictor_col_name not in df_for_regression.columns:
#                 print(f"--- Skipping: Timepoint column '{predictor_col_name}' not found after pivot.")
#                 continue
#
#             timepoint_name = predictor_col_name # שם העמודה (לדוגמה 'b') הוא שם נקודת הזמן
#
#             print(f"\n--- Running Simple Regression: '{y_col}' ~ '{predictor_col_name}' ({PC_COMPONENT} at {timepoint_name}) ---")
#
#             # 4. הכנת X ו-Y, וניקוי ערכים חסרים
#             data_clean = df_for_regression[[predictor_col_name, y_col]].dropna()
#
#             if len(data_clean) < N_SPLITS * 2:
#                 print(f"  Skipping: Not enough data (N={len(data_clean)})")
#                 continue
#
#             # X חייב להיות מטריצה דו-ממדית עבור sklearn
#             X = data_clean[[predictor_col_name]]
#             y = data_clean[y_col]
#
#             # --- 💡 חישוב P-value (P-value עבור מקדם בטא של הרגרסיה)
#             try:
#                 slope, intercept, r_value, p_value, std_err = stats.linregress(X[predictor_col_name], y)
#                 t_stat = slope / std_err
#             except ValueError:
#                 t_stat, p_value, slope, intercept = np.nan, np.nan, np.nan, np.nan
#             # -------------------------------------------------------------
#
#             # 5. שלב א': הערכת המודל (אימות צולב)
#             model_cv = LinearRegression()
#             kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
#             scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}
#
#             cv_scores = cross_validate(model_cv, X, y, cv=kfold, scoring=scoring)
#
#             mean_r2 = np.mean(cv_scores['test_r_squared'])
#             std_r2 = np.std(cv_scores['test_r_squared'])
#             mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))
#             print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")
#
#             # 6. שלב ב': מקדמים סופיים
#             alpha = intercept
#             beta = slope
#             print(f"  Intercept (Alpha): {alpha:.4f} | Slope (Beta): {beta:.4f}")
#             print(f"  P-value (Beta): {p_value:.4f}")
#
#             # ===============================================================
#             #               💡💡💡 תנאי: הצגה רק אם R² חיובי 💡💡💡
#             # ===============================================================
#             if mean_r2 > 0:
#                 print(f"  ✅ R² is positive ({mean_r2:.3f}). Plotting graph.")
#
#                 # 7. שלב ג': שרטוט הגרף (גרף פיזור)
#                 plt.figure(figsize=(8, 6))
#
#                 # גרף פיזור של הנתונים
#                 sns.scatterplot(x=predictor_col_name, y=y_col, data=data_clean, color='darkblue', alpha=0.7)
#
#                 # הוספת קו הרגרסיה
#                 sns.regplot(x=predictor_col_name, y=y_col, data=data_clean, scatter=False, color='red', line_kws={'lw': 2, 'linestyle': '-'})
#
#                 plt.title(f'Prediction: {y_col} by {PC_COMPONENT} at Timepoint {timepoint_name}')
#                 plt.xlabel(f'{PC_COMPONENT} ({timepoint_name})')
#                 plt.ylabel(f'{y_col}')
#
#                 plt.text(0.05, 0.95,
#                          f'$R^2$ (CV) = {mean_r2:.3f}\nSlope (Beta) = {beta:.3f}\n$P$ (Beta) = {p_value:.4f}',
#                          transform=plt.gca().transAxes,
#                          verticalalignment='top',
#                          bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.7))
#
#                 plt.tight_layout()
#
#                 # שמירת הגרף עם שם ייחודי
#                 plot_save_path = os.path.join(PLOTS_SAVE_DIR, f"simple_reg_{y_col}_vs_{PC_COMPONENT}_{predictor_col_name}.png")
#                 plt.savefig(plot_save_path, dpi=100)
#                 plt.close()
#                 print(f"  Plot saved to {plot_save_path}")
#             else:
#                 print(f"  ❌ R² is not positive ({mean_r2:.3f}). Skipping graph plotting.")
#             # ===============================================================
#
#             # 8. שמירת התוצאות לרשימה
#             all_results_list.append({
#                 'Y_Variable': y_col,
#                 'X_Variable (PC1_Timepoint)': predictor_col_name,
#                 'PC_Component': PC_COMPONENT,
#                 'Timepoint': timepoint_name,
#                 'R_Squared_CV': mean_r2,
#                 'R_Squared_Std': std_r2,
#                 'RMSE_CV': mean_rmse,
#                 'Alpha_Intercept': alpha,
#                 'Beta_Slope': beta,
#                 'T_Stat': t_stat,
#                 'P_Value': p_value,
#                 'N': len(y),
#                 'Plotted': mean_r2 > 0
#             })
#
#     print("\n\n--- כל הניתוחים הסתיימו ---")
#
#     # ===================================================================
#     #           חלק 3: יצירת טבלת סיכום
#     # ===================================================================
#
#     if all_results_list:
#         summary_df = pd.DataFrame(all_results_list)
#         # נמיין לפי R^2 יורד
#         summary_df = summary_df.sort_values(by=['Y_Variable', 'R_Squared_CV'], ascending=[True, False])
#
#         print("\n--- טבלת סיכום סופית (רגרסיה פשוטה PC1) ---")
#
#         summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
#         print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
#     else:
#         print("\n--- לא נוצרו תוצאות ---")
#
# else:
#     print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

Loading PC data in Long Format from data/only_Q_outputs/combined/combined_pca_components.csv...
שגיאה: אחד מקובצי הנתונים לא נמצא. ודא נתיבים.
הניתוח לא רץ כי הנתונים לא נטענו כראוי.


## only T1 lasso prediction clinical scores

In [8]:
import pandas as pd
import numpy as np
# ייבוא מודל Lasso במקום LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
import os
import matplotlib.pyplot as plt
# הערה: נשמור על sns לצורך גרף הפיזור, אך קו הרגרסיה של sns.regplot יתבסס על מודל ליניארי קלאסי (לא מרוגולר)
import seaborn as sns
# נסיר את stats כי לא נחשב P-value קלאסי
# from scipy import stats

# ===================================================================
#           חלק 1: הגדרות נתיבים ופרמטרים
# ===================================================================

# נתיב הקובץ - הקובץ מכיל את כל הנתונים הנדרשים (X ו-Y)
DATA_PATH = "data/merged_no_nan.csv"

# תיקייה לשמירת כל הגרפים
PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/LASSO_REGRESSION_CUSTOM_PREDICTORS_plots" # שונה
# קובץ לשמירת טבלת הסיכום
SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/LASSO_REGRESSION_CUSTOM_PREDICTORS_summary_table.csv" # שונה

# 💡💡💡
# --- הגדר כאן את הפרמטרים ---
# 💡💡💡

# 1. רשימת המשתנים *הקבועים* שאתה רוצה לנבא (Y) - נשתמש בשמות המתוקנים (lowercase)
Y_VARIABLES_TO_PREDICT = [
    'b_des_average',
    'after_des_total',
    "b_ders_total",
    "after_ders_total",
    "diff_ders",
    "diff_des"
]

# 2. רשימת המשתנים המנבאים (X) שאתה רוצה להשתמש בהם
X_PREDICTORS = [
    'dmn_gm',
    'fpn_gm',
    'lim_gm',
    'smn_gm',
    'van_gm',
    'vis_gm'
]

# 3. פרמטר רגולריזציה של לאסו (Alpha)
# 💡 אלפא = 0 מחזיר רגרסיה ליניארית רגילה. ערך קטן (כמו 0.01) מפעיל רגולריזציה עדינה.
LASSO_ALPHA = 0.01

# 4. הגדרות לאימות צולב
N_SPLITS = 5
RANDOM_STATE = 42

df_for_regression = None

try:
    # ===================================================================
    #           חלק 1: הכנת הנתונים (טעינה וניקוי)
    # ===================================================================

    print(f"Loading merged data (X and Y variables) from {DATA_PATH}...")

    # טוענים את כל העמודות הדרושות: subject_code (תוקן ל-lowercase), X ו-Y
    columns_to_load = ['subject_code'] + Y_VARIABLES_TO_PREDICT + X_PREDICTORS

    # טעינת הנתונים.
    df_for_regression = pd.read_csv(DATA_PATH, usecols=lambda c: c in columns_to_load)

    # המרה ל-float64 לכל העמודות הרלוונטיות
    all_var_cols = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    for col in all_var_cols:
        if col in df_for_regression.columns:
            df_for_regression[col] = pd.to_numeric(df_for_regression[col], errors='coerce').astype(np.float64)

    print("--- Data successfully loaded (ready for regression) ---")
    print(df_for_regression.head())

    # יצירת תיקיית הפלט לגרפים
    os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)

except FileNotFoundError:
    print(f"שגיאה: קובץ הנתונים לא נמצא. ודא נתיב: {DATA_PATH}")
    df_for_regression = None
except Exception as e:
    print(f"אירעה שגיאה: {e}")
    df_for_regression = None

# ===================================================================
#           חלק 2: הרצת הניתוח בלולאה (רגרסיית לאסו ו-CV)
# ===================================================================

all_results_list = []

if df_for_regression is not None:

    # --- התחלת הלולאות ---
    for y_col in Y_VARIABLES_TO_PREDICT:
        print(f"\n=======================================================")
        print(f"  Processing Y-Variable: {y_col}")
        print(f"=======================================================")

        # *** הלולאה רצה על כל משתנה מנבא (X) מותאם אישית ***
        for predictor_col_name in X_PREDICTORS:

            if predictor_col_name not in df_for_regression.columns:
                print(f"--- Skipping: Predictor column '{predictor_col_name}' not found in data.")
                continue

            print(f"\n--- Running Simple LASSO Regression (Alpha={LASSO_ALPHA}): '{y_col}' ~ '{predictor_col_name}' ---")

            # 4. הכנת X ו-Y, וניקוי ערכים חסרים (NaN)
            data_clean = df_for_regression[[predictor_col_name, y_col]].dropna()

            if len(data_clean) < N_SPLITS * 2:
                print(f"  Skipping: Not enough data (N={len(data_clean)}). Need at least {N_SPLITS*2} for CV.")
                continue

            # X חייב להיות מטריצה דו-ממדית עבור sklearn
            X = data_clean[[predictor_col_name]]
            y = data_clean[y_col]

            # --- 💡 שלב 5א: הערכת המודל (אימות צולב) באמצעות Lasso
            # שימוש ב-Lasso עם הפרמטר Alpha
            model_cv = Lasso(alpha=LASSO_ALPHA, random_state=RANDOM_STATE, max_iter=10000)
            kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
            scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}

            cv_scores = cross_validate(model_cv, X, y, cv=kfold, scoring=scoring)

            mean_r2 = np.mean(cv_scores['test_r_squared'])
            std_r2 = np.std(cv_scores['test_r_squared'])
            mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))
            print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")

            # -------------------------------------------------------------
            # --- שלב 5ב: אימון המודל על כל הנתונים כדי לקבל מקדמים סופיים ---
            final_model = Lasso(alpha=LASSO_ALPHA, random_state=RANDOM_STATE, max_iter=10000)
            final_model.fit(X, y)

            # מקדמים
            alpha_intercept = final_model.intercept_
            beta_slope = final_model.coef_[0]

            # P-value: לא מחושב עבור מודלים מרוגולרים. נשתמש ב-NaN.
            t_stat, p_value = np.nan, np.nan

            print(f"  Intercept (Alpha): {alpha_intercept:.4f} | Slope (Beta, LASSO): {beta_slope:.4f}")
            print(f"  Note: P-value is not calculated for Regularized models.")
            # -------------------------------------------------------------

            # ===============================================================
            #               💡💡💡 תנאי: הצגה רק אם R² חיובי 💡💡💡
            # ===============================================================
            if mean_r2 > 0:
                print(f"  ✅ R² is positive ({mean_r2:.3f}). Plotting graph.")

                # 7. שלב ג': שרטוט הגרף (גרף פיזור)
                plt.figure(figsize=(8, 6))

                # גרף פיזור של הנתונים
                sns.scatterplot(x=predictor_col_name, y=y_col, data=data_clean, color='darkblue', alpha=0.7)

                # הוספת קו הרגרסיה של ה-LASSO:
                # נשתמש בנוסחת הקו (y = Beta*X + Alpha)
                x_fit = np.linspace(X[predictor_col_name].min(), X[predictor_col_name].max(), 100).reshape(-1, 1)
                y_fit = final_model.predict(x_fit)

                plt.plot(x_fit, y_fit, color='red', linestyle='-', linewidth=2, label=f'LASSO Fit (Alpha={LASSO_ALPHA})')

                plt.title(f'LASSO Prediction: {y_col} by {predictor_col_name}')
                plt.xlabel(f'{predictor_col_name}')
                plt.ylabel(f'{y_col}')

                plt.text(0.05, 0.95,
                         f'$R^2$ (CV) = {mean_r2:.3f}\nSlope (Beta, LASSO) = {beta_slope:.3f}\nAlpha (LASSO) = {LASSO_ALPHA}',
                         transform=plt.gca().transAxes,
                         verticalalignment='top',
                         bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.7))

                plt.tight_layout()

                # שמירת הגרף עם שם ייחודי
                plot_save_path = os.path.join(PLOTS_SAVE_DIR, f"lasso_reg_{y_col}_vs_{predictor_col_name}.png")
                plt.savefig(plot_save_path, dpi=100)
                plt.close()
                print(f"  Plot saved to {plot_save_path}")
            else:
                print(f"  ❌ R² is not positive ({mean_r2:.3f}). Skipping graph plotting.")
            # ===============================================================

            # 8. שמירת התוצאות לרשימה
            all_results_list.append({
                'Y_Variable': y_col,
                'X_Variable': predictor_col_name,
                'Model': 'LASSO',
                'Alpha_Lasso': LASSO_ALPHA,
                'R_Squared_CV': mean_r2,
                'R_Squared_Std': std_r2,
                'RMSE_CV': mean_rmse,
                'Alpha_Intercept': alpha_intercept,
                'Beta_Slope': beta_slope,
                'T_Stat': t_stat, # NaN
                'P_Value': p_value, # NaN
                'N': len(y),
                'Plotted': mean_r2 > 0
            })

    print("\n\n--- כל הניתוחים הסתיימו ---")

    # ===================================================================
    #           חלק 3: יצירת טבלת סיכום
    # ===================================================================

    if all_results_list:
        summary_df = pd.DataFrame(all_results_list)
        # נמיין לפי R^2 יורד
        summary_df = summary_df.sort_values(by=['Y_Variable', 'R_Squared_CV'], ascending=[True, False])

        print("\n--- טבלת סיכום סופית (רגרסיית לאסו מנבאים מותאמים אישית) ---")

        summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
        print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
    else:
        print("\n--- לא נוצרו תוצאות ---")

else:
    print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

Loading merged data (X and Y variables) from data/merged_no_nan.csv...
--- Data successfully loaded (ready for regression) ---
  subject_code  b_ders_total  b_des_average  after_des_total  \
0        CT003          68.0       2.142857         0.000000   
1        CT004          67.0       4.642857         7.500000   
2        CT005          95.0       7.142857         5.000000   
3        CT007          70.0      23.928571        10.357143   
4        CT008          69.0       2.500000         1.785714   

   after_ders_total   diff_des  diff_ders    dmn_gm   fpn_gm   lim_gm  \
0              91.0  -2.142857       23.0  159872.0  47224.0  66238.0   
1              69.0   2.857143        2.0  172948.0  54126.0  67313.0   
2              87.0  -2.142857       -8.0  127745.0  35090.0  55938.0   
3              72.0 -13.571429        2.0  142435.0  42930.0  59068.0   
4              47.0  -0.714286      -22.0  143723.0  38971.0  53522.0   

    smn_gm   van_gm   vis_gm  
0  88987.0  50939.

## lasso t1 network and pc's

In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# ===================================================================
#           חלק 1: הגדרות נתיבים ופרמטרים
# ===================================================================

# נתיב הקובץ הראשי (X ו-Y)
DATA_PATH = "data/merged_no_nan.csv"

# הגדרות קובץ ה-PC
PC_DATA_PATH = "data/only_Q_outputs/combined/combined_pca_components.csv"

# תיקיות שמירה
PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/LASSO_MULTIPLE_REGRESSION_PC_PREDICTORS_plots"
SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/LASSO_MULTIPLE_REGRESSION_PC_PREDICTORS_summary_table.csv"

# 💡💡💡
# --- הגדר כאן את הפרמטרים ---
# 💡💡💡

# 1. רשימת המשתנים לניבוי (Y)
Y_VARIABLES_TO_PREDICT = [
    'b_des_average',
    'after_des_total',
    "b_ders_total",
    "after_ders_total",
    "diff_ders",
    "diff_des"
]

# 2. רשימת המשתנים המנבאים (X) המקורית - יתווספו אליה רכיבי ה-PC בהמשך!
X_PREDICTORS = [
    'dmn_gm',
    'fpn_gm',
    'lim_gm',
    'smn_gm',
    'van_gm',
    'vis_gm'
]

# 3. פרמטר רגולריזציה של לאסו
LASSO_ALPHA = 0.01

# 4. הגדרות לאימות צולב
N_SPLITS = 5
RANDOM_STATE = 42

df_for_regression = None

try:
    # ===================================================================
    #           חלק 1: הכנת הנתונים (טעינה וניקוי)
    # ===================================================================

    print(f"Loading main data (X and Y variables) from {DATA_PATH}...")

    # טוענים את כל העמודות הדרושות: subject_code, X ו-Y, בתוספת timepoint לצורך סינון
    columns_to_load = ['subject_code', 'timepoint'] + Y_VARIABLES_TO_PREDICT + X_PREDICTORS

    df_for_regression = pd.read_csv(DATA_PATH, usecols=lambda c: c in columns_to_load)
    df_for_regression.columns = [c.lower() for c in df_for_regression.columns] # ודא שכל העמודות ב-lowercase

    # *** שינוי 1: הסרת עמודת timepoint מה-DataFrame הראשי ***
    if 'timepoint' in df_for_regression.columns:
        df_for_regression = df_for_regression.drop(columns=['timepoint'])
        print("--- Removed 'timepoint' column from main data. ---")

    # המרה ל-float64 לכל העמודות הרלוונטיות
    all_var_cols = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    for col in all_var_cols:
        if col in df_for_regression.columns:
            df_for_regression[col] = pd.to_numeric(df_for_regression[col], errors='coerce').astype(np.float64)

    print("--- Main data successfully loaded ---")

    # -------------------------------------------------------------------
    #           חלק 1.5: טעינת רכיבי PC ומיזוגם (בפורמט רחב)
    # -------------------------------------------------------------------
    print(f"\nLoading ALL PC data (Wide Format) from {PC_DATA_PATH}...")

    df_pc_wide = pd.read_csv(PC_DATA_PATH)
    df_pc_wide.columns = [c.lower() for c in df_pc_wide.columns]

    subject_id_col = 'subject_code'

    # *** שינוי 2: סינון נתוני ה-PC לפי timepoint (אם העמודה קיימת) ***
    if 'timepoint' in df_pc_wide.columns:
        initial_pc_count = len(df_pc_wide)

        # סינון לשורות המכילות 'b/befpr' (ב-lowercase)
        df_pc_wide = df_pc_wide[df_pc_wide['timepoint'].str.contains('b', na=False, case=False)]

        # הסרת עמודת timepoint לאחר הסינון, כדי למנוע הוספתה כמשתנה מנבא
        df_pc_wide = df_pc_wide.drop(columns=['timepoint'])

        print(f"--- PC data filtered by timepoint='b/befpr'. Rows reduced from {initial_pc_count} to {len(df_pc_wide)} ---")
    else:
        print("--- 'timepoint' column not found in PC file. No filtering applied. ---")


    # שמירת רשימת שמות ה-PC שנוצרו (כל עמודה שהיא לא subject_code)
    pc_columns_list = [col for col in df_pc_wide.columns if col != subject_id_col]

    # המרה ל-float64 עבור עמודות ה-PC
    for col in pc_columns_list:
        df_pc_wide[col] = pd.to_numeric(df_pc_wide[col], errors='coerce').astype(np.float64)

    # מיזוג ה-PC עם ה-DataFrame הראשי
    df_for_regression = pd.merge(
        df_for_regression,
        df_pc_wide,
        on=subject_id_col,
        how='left'
    )
    print(f"--- All PC components successfully merged. New shape: {df_for_regression.shape} ---")

    # -------------------------------------------------------------------
    #           עדכון רשימת המנבאים (X) לכלול את כל ה-PC
    # -------------------------------------------------------------------
    X_PREDICTORS.extend(pc_columns_list)
    print(f"\nUpdated X Predictors (Total {len(X_PREDICTORS)}): {X_PREDICTORS}")

    # יצירת תיקיית הפלט לגרפים
    os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)

except FileNotFoundError:
    print(f"שגיאה: קובץ הנתונים הראשי או קובץ ה-PC לא נמצא. ודא נתיבים: {DATA_PATH} ו- {PC_DATA_PATH}")
    df_for_regression = None
except Exception as e:
    print(f"אירעה שגיאה: {e}")
    df_for_regression = None

# ===================================================================
#           חלק 2: הרצת הניתוח בלולאה (רגרסיית LASSO מרובת משתנים)
# ===================================================================

all_results_list = []

if df_for_regression is not None and not df_for_regression.empty:

    # 1. ניקוי הנתונים החסרים (NaN) מראש על כל ה-X וכל ה-Y
    columns_to_keep = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    df_clean_data = df_for_regression[columns_to_keep].dropna()

    if df_clean_data.empty:
        print("--- Error: No complete data rows remaining after dropping NaNs. Analysis stopped. ---")
    else:
        print(f"\n--- Data cleaned. N={len(df_clean_data)} subjects used in all models. ---")

        # X המשותף לכל הרגרסיות (כל המנבאים יחד)
        X_full = df_clean_data[X_PREDICTORS]

        # --- התחלת הלולאה על משתני ה-Y בלבד ---
        for y_col in Y_VARIABLES_TO_PREDICT:
            print(f"\n=======================================================")
            print(f"  Processing Y-Variable: {y_col} (LASSO Multiple Regression)")
            print(f"=======================================================")

            y = df_clean_data[y_col]

            # -------------------------------------------------------------
            # --- שלב 5א: הערכת המודל (אימות צולב) באמצעות Lasso
            # -------------------------------------------------------------
            model_cv = Lasso(alpha=LASSO_ALPHA, random_state=RANDOM_STATE, max_iter=10000)
            kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
            scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}

            cv_scores = cross_validate(model_cv, X_full, y, cv=kfold, scoring=scoring)

            mean_r2 = np.mean(cv_scores['test_r_squared'])
            std_r2 = np.std(cv_scores['test_r_squared'])
            mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))

            print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")

            # -------------------------------------------------------------
            # --- שלב 5ב: אימון המודל על כל הנתונים כדי לקבל מקדמים סופיים ---
            # -------------------------------------------------------------
            final_model = Lasso(alpha=LASSO_ALPHA, random_state=RANDOM_STATE, max_iter=10000)
            final_model.fit(X_full, y)

            alpha_intercept = final_model.intercept_
            betas = final_model.coef_

            print(f"  Intercept (Alpha): {alpha_intercept:.4f}")

            # ===============================================================
            #               שמירת תוצאות (מעבר על כל מקדם בנפרד)
            # ===============================================================

            for i, predictor_name in enumerate(X_PREDICTORS):
                all_results_list.append({
                    'Y_Variable': y_col,
                    'X_Variable': predictor_name,
                    'Model': 'LASSO_Multiple',
                    'Alpha_Lasso_Reg': LASSO_ALPHA,
                    'R_Squared_CV': mean_r2,
                    'R_Squared_Std': std_r2,
                    'RMSE_CV': mean_rmse,
                    'Alpha_Intercept': alpha_intercept,
                    'Beta_Slope': betas[i],
                    'T_Stat': np.nan,
                    'P_Value': np.nan,
                    'N': len(y),
                    'Plotted': False
                })
                if abs(betas[i]) > 1e-9:
                    print(f"    -> Beta({predictor_name}): {betas[i]:.4f}")


        print("\n\n--- כל ניתוחי ה-LASSO המרובים הסתיימו ---")

        # ===================================================================
        #           חלק 3: יצירת טבלת סיכום
        # ===================================================================

        if all_results_list:
            summary_df = pd.DataFrame(all_results_list)
            summary_df = summary_df.sort_values(by=['Y_Variable', 'Beta_Slope'], ascending=[True, False])

            print("\n--- טבלת סיכום סופית (רגרסיית LASSO מרובת משתנים) ---")

            summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
            print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
        else:
            print("\n--- לא נוצרו תוצאות ---")

else:
    print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

Loading main data (X and Y variables) from data/merged_no_nan.csv...
--- Main data successfully loaded ---

Loading ALL PC data (Wide Format) from data/only_Q_outputs/combined/combined_pca_components.csv...
שגיאה: קובץ הנתונים הראשי או קובץ ה-PC לא נמצא. ודא נתיבים: data/merged_no_nan.csv ו- data/only_Q_outputs/combined/combined_pca_components.csv
הניתוח לא רץ כי הנתונים לא נטענו כראוי.


## lasso t1 lobes and pc's


In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# ===================================================================
#           חלק 1: הגדרות נתיבים ופרמטרים
# ===================================================================

# נתיב הקובץ הראשי (X ו-Y)
DATA_PATH = "data/merged_no_nan.csv"

# הגדרות קובץ ה-PC
PC_DATA_PATH = "data/only_Q_outputs/combined/combined_pca_components.csv"

# תיקיות שמירה
PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/LASSO_MULTIPLE_REGRESSION_PC_PREDICTORS_plots"
SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/LASSO_MULTIPLE_REGRESSION_PC_PREDICTORS_summary_table.csv"

# 💡💡💡
# --- הגדר כאן את הפרמטרים ---
# 💡💡💡

# 1. רשימת המשתנים לניבוי (Y)
Y_VARIABLES_TO_PREDICT = [
    'b_des_average',
    'after_des_total',
    "b_ders_total",
    "after_ders_total",
    "diff_ders",
    "diff_des"
]

# 2. רשימת המשתנים המנבאים (X) המקורית - יתווספו אליה רכיבי ה-PC בהמשך!
X_PREDICTORS = [ "cingulate","frontal","insula","occipital","parietal","temporal"]

# 3. פרמטר רגולריזציה של לאסו
LASSO_ALPHA = 0.01

# 4. הגדרות לאימות צולב
N_SPLITS = 5
RANDOM_STATE = 42

df_for_regression = None

try:
    # ===================================================================
    #           חלק 1: הכנת הנתונים (טעינה וניקוי)
    # ===================================================================

    print(f"Loading main data (X and Y variables) from {DATA_PATH}...")

    # טוענים את כל העמודות הדרושות: subject_code, X ו-Y, בתוספת timepoint לצורך סינון
    columns_to_load = ['subject_code', 'timepoint'] + Y_VARIABLES_TO_PREDICT + X_PREDICTORS

    df_for_regression = pd.read_csv(DATA_PATH, usecols=lambda c: c in columns_to_load)
    df_for_regression.columns = [c.lower() for c in df_for_regression.columns] # ודא שכל העמודות ב-lowercase

    # *** שינוי 1: הסרת עמודת timepoint מה-DataFrame הראשי ***
    if 'timepoint' in df_for_regression.columns:
        df_for_regression = df_for_regression.drop(columns=['timepoint'])
        print("--- Removed 'timepoint' column from main data. ---")

    # המרה ל-float64 לכל העמודות הרלוונטיות
    all_var_cols = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    for col in all_var_cols:
        if col in df_for_regression.columns:
            df_for_regression[col] = pd.to_numeric(df_for_regression[col], errors='coerce').astype(np.float64)

    print("--- Main data successfully loaded ---")

    # -------------------------------------------------------------------
    #           חלק 1.5: טעינת רכיבי PC ומיזוגם (בפורמט רחב)
    # -------------------------------------------------------------------
    print(f"\nLoading ALL PC data (Wide Format) from {PC_DATA_PATH}...")

    df_pc_wide = pd.read_csv(PC_DATA_PATH)
    df_pc_wide.columns = [c.lower() for c in df_pc_wide.columns]

    subject_id_col = 'subject_code'

    # *** שינוי 2: סינון נתוני ה-PC לפי timepoint (אם העמודה קיימת) ***
    if 'timepoint' in df_pc_wide.columns:
        initial_pc_count = len(df_pc_wide)

        # סינון לשורות המכילות 'b/befpr' (ב-lowercase)
        df_pc_wide = df_pc_wide[df_pc_wide['timepoint'].str.contains('b', na=False, case=False)]

        # הסרת עמודת timepoint לאחר הסינון, כדי למנוע הוספתה כמשתנה מנבא
        df_pc_wide = df_pc_wide.drop(columns=['timepoint'])

        print(f"--- PC data filtered by timepoint='b/befpr'. Rows reduced from {initial_pc_count} to {len(df_pc_wide)} ---")
    else:
        print("--- 'timepoint' column not found in PC file. No filtering applied. ---")


    # שמירת רשימת שמות ה-PC שנוצרו (כל עמודה שהיא לא subject_code)
    pc_columns_list = [col for col in df_pc_wide.columns if col != subject_id_col]

    # המרה ל-float64 עבור עמודות ה-PC
    for col in pc_columns_list:
        df_pc_wide[col] = pd.to_numeric(df_pc_wide[col], errors='coerce').astype(np.float64)

    # מיזוג ה-PC עם ה-DataFrame הראשי
    df_for_regression = pd.merge(
        df_for_regression,
        df_pc_wide,
        on=subject_id_col,
        how='left'
    )
    print(f"--- All PC components successfully merged. New shape: {df_for_regression.shape} ---")

    # -------------------------------------------------------------------
    #           עדכון רשימת המנבאים (X) לכלול את כל ה-PC
    # -------------------------------------------------------------------
    X_PREDICTORS.extend(pc_columns_list)
    print(f"\nUpdated X Predictors (Total {len(X_PREDICTORS)}): {X_PREDICTORS}")

    # יצירת תיקיית הפלט לגרפים
    os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)

except FileNotFoundError:
    print(f"שגיאה: קובץ הנתונים הראשי או קובץ ה-PC לא נמצא. ודא נתיבים: {DATA_PATH} ו- {PC_DATA_PATH}")
    df_for_regression = None
except Exception as e:
    print(f"אירעה שגיאה: {e}")
    df_for_regression = None

# ===================================================================
#           חלק 2: הרצת הניתוח בלולאה (רגרסיית LASSO מרובת משתנים)
# ===================================================================

all_results_list = []

if df_for_regression is not None and not df_for_regression.empty:

    # 1. ניקוי הנתונים החסרים (NaN) מראש על כל ה-X וכל ה-Y
    columns_to_keep = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    df_clean_data = df_for_regression[columns_to_keep].dropna()

    if df_clean_data.empty:
        print("--- Error: No complete data rows remaining after dropping NaNs. Analysis stopped. ---")
    else:
        print(f"\n--- Data cleaned. N={len(df_clean_data)} subjects used in all models. ---")

        # X המשותף לכל הרגרסיות (כל המנבאים יחד)
        X_full = df_clean_data[X_PREDICTORS]

        # --- התחלת הלולאה על משתני ה-Y בלבד ---
        for y_col in Y_VARIABLES_TO_PREDICT:
            print(f"\n=======================================================")
            print(f"  Processing Y-Variable: {y_col} (LASSO Multiple Regression)")
            print(f"=======================================================")

            y = df_clean_data[y_col]

            # -------------------------------------------------------------
            # --- שלב 5א: הערכת המודל (אימות צולב) באמצעות Lasso
            # -------------------------------------------------------------
            model_cv = Lasso(alpha=LASSO_ALPHA, random_state=RANDOM_STATE, max_iter=10000)
            kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
            scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}

            cv_scores = cross_validate(model_cv, X_full, y, cv=kfold, scoring=scoring)

            mean_r2 = np.mean(cv_scores['test_r_squared'])
            std_r2 = np.std(cv_scores['test_r_squared'])
            mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))

            print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")

            # -------------------------------------------------------------
            # --- שלב 5ב: אימון המודל על כל הנתונים כדי לקבל מקדמים סופיים ---
            # -------------------------------------------------------------
            final_model = Lasso(alpha=LASSO_ALPHA, random_state=RANDOM_STATE, max_iter=10000)
            final_model.fit(X_full, y)

            alpha_intercept = final_model.intercept_
            betas = final_model.coef_

            print(f"  Intercept (Alpha): {alpha_intercept:.4f}")

            # ===============================================================
            #               שמירת תוצאות (מעבר על כל מקדם בנפרד)
            # ===============================================================

            for i, predictor_name in enumerate(X_PREDICTORS):
                all_results_list.append({
                    'Y_Variable': y_col,
                    'X_Variable': predictor_name,
                    'Model': 'LASSO_Multiple',
                    'Alpha_Lasso_Reg': LASSO_ALPHA,
                    'R_Squared_CV': mean_r2,
                    'R_Squared_Std': std_r2,
                    'RMSE_CV': mean_rmse,
                    'Alpha_Intercept': alpha_intercept,
                    'Beta_Slope': betas[i],
                    'T_Stat': np.nan,
                    'P_Value': np.nan,
                    'N': len(y),
                    'Plotted': False
                })
                if abs(betas[i]) > 1e-9:
                    print(f"    -> Beta({predictor_name}): {betas[i]:.4f}")


        print("\n\n--- כל ניתוחי ה-LASSO המרובים הסתיימו ---")

        # ===================================================================
        #           חלק 3: יצירת טבלת סיכום
        # ===================================================================

        if all_results_list:
            summary_df = pd.DataFrame(all_results_list)
            summary_df = summary_df.sort_values(by=['Y_Variable', 'Beta_Slope'], ascending=[True, False])

            print("\n--- טבלת סיכום סופית (רגרסיית LASSO מרובת משתנים) ---")

            summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
            print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
        else:
            print("\n--- לא נוצרו תוצאות ---")

else:
    print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

Loading main data (X and Y variables) from data/merged_no_nan.csv...
--- Main data successfully loaded ---

Loading ALL PC data (Wide Format) from data/only_Q_outputs/combined/combined_pca_components.csv...
שגיאה: קובץ הנתונים הראשי או קובץ ה-PC לא נמצא. ודא נתיבים: data/merged_no_nan.csv ו- data/only_Q_outputs/combined/combined_pca_components.csv
הניתוח לא רץ כי הנתונים לא נטענו כראוי.


In [11]:
import pandas as pd
import numpy as np
# 💡 שינוי: ייבוא Ridge במקום Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# ===================================================================
#           חלק 1: הגדרות נתיבים ופרמטרים
# ===================================================================

# נתיב הקובץ הראשי (X ו-Y)
DATA_PATH = "data/merged_no_nan.csv"

# הגדרות קובץ ה-PC
PC_DATA_PATH = "data/only_Q_outputs/combined/combined_pca_components.csv"

# תיקיות שמירה
# 💡 עדכון שמות הפלט ל-RIDGE
PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/RIDGE_MULTIPLE_REGRESSION_PC_PREDICTORS_plots"
SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/RIDGE_MULTIPLE_REGRESSION_PC_PREDICTORS_summary_table.csv"

# 💡💡💡
# --- הגדר כאן את הפרמטרים ---
# 💡💡💡

# 1. רשימת המשתנים לניבוי (Y)
Y_VARIABLES_TO_PREDICT = [
    'b_des_average',
    'after_des_total',
    "b_ders_total",
    "after_ders_total",
    "diff_ders",
    "diff_des"
]

# 2. רשימת המשתנים המנבאים (X) המקורית - יתווספו אליה רכיבי ה-PC בהמשך!
X_PREDICTORS = [
    'dmn_gm',
    'fpn_gm',
    'lim_gm',
    'smn_gm',
    'van_gm',
    'vis_gm'
]

# 3. פרמטר רגולריזציה (נקרא גם Alpha או Lambda)
RIDGE_ALPHA = 0.01

# 4. הגדרות לאימות צולב
N_SPLITS = 5
RANDOM_STATE = 42

df_for_regression = None

try:
    # ===================================================================
    #           חלק 1: הכנת הנתונים (טעינה וניקוי) - ללא שינוי
    # ===================================================================

    print(f"Loading main data (X and Y variables) from {DATA_PATH}...")
    columns_to_load = ['subject_code', 'timepoint'] + Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    df_for_regression = pd.read_csv(DATA_PATH, usecols=lambda c: c in columns_to_load)
    df_for_regression.columns = [c.lower() for c in df_for_regression.columns]

    # הסרת עמודת timepoint מה-DataFrame הראשי
    if 'timepoint' in df_for_regression.columns:
        df_for_regression = df_for_regression.drop(columns=['timepoint'])
        print("--- Removed 'timepoint' column from main data. ---")

    all_var_cols = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    for col in all_var_cols:
        if col in df_for_regression.columns:
            df_for_regression[col] = pd.to_numeric(df_for_regression[col], errors='coerce').astype(np.float64)

    print("--- Main data successfully loaded ---")

    # -------------------------------------------------------------------
    #           חלק 1.5: טעינת רכיבי PC ומיזוגם (בפורמט רחב) - ללא שינוי
    # -------------------------------------------------------------------
    print(f"\nLoading ALL PC data (Wide Format) from {PC_DATA_PATH}...")

    df_pc_wide = pd.read_csv(PC_DATA_PATH)
    df_pc_wide.columns = [c.lower() for c in df_pc_wide.columns]

    subject_id_col = 'subject_code'

    # סינון נתוני ה-PC לפי timepoint (אם העמודה קיימת)
    if 'timepoint' in df_pc_wide.columns:
        initial_pc_count = len(df_pc_wide)
        df_pc_wide = df_pc_wide[df_pc_wide['timepoint'].str.contains('b', na=False, case=False)]
        df_pc_wide = df_pc_wide.drop(columns=['timepoint'])
        print(f"--- PC data filtered by timepoint='b/befpr'. Rows reduced from {initial_pc_count} to {len(df_pc_wide)} ---")
    else:
        print("--- 'timepoint' column not found in PC file. No filtering applied. ---")


    pc_columns_list = [col for col in df_pc_wide.columns if col != subject_id_col]

    for col in pc_columns_list:
        df_pc_wide[col] = pd.to_numeric(df_pc_wide[col], errors='coerce').astype(np.float64)

    df_for_regression = pd.merge(
        df_for_regression,
        df_pc_wide,
        on=subject_id_col,
        how='left'
    )
    print(f"--- All PC components successfully merged. New shape: {df_for_regression.shape} ---")

    X_PREDICTORS.extend(pc_columns_list)
    print(f"\nUpdated X Predictors (Total {len(X_PREDICTORS)}): {X_PREDICTORS}")

    os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)

except FileNotFoundError:
    print(f"שגיאה: קובץ הנתונים הראשי או קובץ ה-PC לא נמצא. ודא נתיבים: {DATA_PATH} ו- {PC_DATA_PATH}")
    df_for_regression = None
except Exception as e:
    print(f"אירעה שגיאה: {e}")
    df_for_regression = None

# ===================================================================
#           חלק 2: הרצת הניתוח בלולאה (רגרסיית RIDGE מרובת משתנים)
# ===================================================================

all_results_list = []

if df_for_regression is not None and not df_for_regression.empty:

    columns_to_keep = Y_VARIABLES_TO_PREDICT + X_PREDICTORS
    df_clean_data = df_for_regression[columns_to_keep].dropna()
    df_for_regression[columns_to_keep].to_csv('data/only_Q_outputs/combined/merged_pc_t1_clinical.csv')
    if df_clean_data.empty:
        print("--- Error: No complete data rows remaining after dropping NaNs. Analysis stopped. ---")
    else:
        print(f"\n--- Data cleaned. N={len(df_clean_data)} subjects used in all models. ---")

        X_full = df_clean_data[X_PREDICTORS]

        for y_col in Y_VARIABLES_TO_PREDICT:
            print(f"\n=======================================================")
            print(f"  Processing Y-Variable: {y_col} (RIDGE Multiple Regression)")
            print(f"=======================================================")

            y = df_clean_data[y_col]

            # -------------------------------------------------------------
            # --- שלב 5א: הערכת המודל (אימות צולב) באמצעות RIDGE
            # -------------------------------------------------------------
            # 💡 שינוי: שימוש ב-Ridge במקום Lasso
            model_cv = Ridge(alpha=RIDGE_ALPHA, random_state=RANDOM_STATE)
            kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
            scoring = {'r_squared': 'r2', 'neg_mse': 'neg_mean_squared_error'}

            cv_scores = cross_validate(model_cv, X_full, y, cv=kfold, scoring=scoring)

            mean_r2 = np.mean(cv_scores['test_r_squared'])
            std_r2 = np.std(cv_scores['test_r_squared'])
            mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))

            print(f"  R² (CV): {mean_r2:.3f} | RMSE (CV): {mean_rmse:.3f} | N = {len(y)}")

            # -------------------------------------------------------------
            # --- שלב 5ב: אימון המודל על כל הנתונים כדי לקבל מקדמים סופיים ---
            # -------------------------------------------------------------
            # 💡 שינוי: שימוש ב-Ridge במקום Lasso
            final_model = Ridge(alpha=RIDGE_ALPHA, random_state=RANDOM_STATE)
            final_model.fit(X_full, y)

            alpha_intercept = final_model.intercept_
            betas = final_model.coef_

            print(f"  Intercept (Alpha): {alpha_intercept:.4f}")

            # ===============================================================
            #               שמירת תוצאות
            # ===============================================================

            for i, predictor_name in enumerate(X_PREDICTORS):
                all_results_list.append({
                    'Y_Variable': y_col,
                    'X_Variable': predictor_name,
                    # 💡 עדכון: שם המודל
                    'Model': 'RIDGE_Multiple',
                    'Alpha_Ridge_Reg': RIDGE_ALPHA,
                    'R_Squared_CV': mean_r2,
                    'R_Squared_Std': std_r2,
                    'RMSE_CV': mean_rmse,
                    'Alpha_Intercept': alpha_intercept,
                    'Beta_Slope': betas[i],
                    'T_Stat': np.nan,
                    'P_Value': np.nan,
                    'N': len(y),
                    'Plotted': False
                })
                # הדפסת מקדמים משמעותיים
                if abs(betas[i]) > 1e-9:
                    print(f"    -> Beta({predictor_name}): {betas[i]:.4f}")


        print("\n\n--- כל ניתוחי ה-RIDGE המרובים הסתיימו ---")

        # ===================================================================
        #           חלק 3: יצירת טבלת סיכום
        # ===================================================================

        if all_results_list:
            summary_df = pd.DataFrame(all_results_list)
            summary_df = summary_df.sort_values(by=['Y_Variable', 'Beta_Slope'], ascending=[True, False])

            print("\n--- טבלת סיכום סופית (רגרסיית RIDGE מרובת משתנים) ---")

            summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False, encoding='utf-8-sig')
            print(f"\n✅ טבלת הסיכום נשמרה ב: {SUMMARY_TABLE_SAVE_PATH}")
        else:
            print("\n--- לא נוצרו תוצאות ---")

else:
    print("הניתוח לא רץ כי הנתונים לא נטענו כראוי.")

Loading main data (X and Y variables) from data/merged_no_nan.csv...
--- Main data successfully loaded ---

Loading ALL PC data (Wide Format) from data/only_Q_outputs/combined/combined_pca_components.csv...
שגיאה: קובץ הנתונים הראשי או קובץ ה-PC לא נמצא. ודא נתיבים: data/merged_no_nan.csv ו- data/only_Q_outputs/combined/combined_pca_components.csv
הניתוח לא רץ כי הנתונים לא נטענו כראוי.


## clusters with lasso regrssion


In [12]:
# import pandas as pd
# import numpy as np
# from sklearn.linear_model import LassoCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import KFold, cross_validate
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
#
# # ===================================================================
# #           PART 1: Settings and Parameters
# # ===================================================================
#
# # File Paths
# CLUSTER_DATA_WIDE_PATH = "data/only_Q_outputs/combined/timepoints_file_inverted_2.csv"
# raw_data_path = "data/only_Q_outputs/combined/regression_parameters.csv"
#
# # Output Directories
# PLOTS_SAVE_DIR = "data/only_Q_outputs/combined/regression_LASSO_Cluster_SingleTimepoint_plots"
# SUMMARY_TABLE_SAVE_PATH = "data/only_Q_outputs/combined/regression_LASSO_Cluster_SingleTimepoint_summary.csv"
#
# # 💡PARAMETERS 💡
#
# # 1. Y Variables
# Y_VARIABLES_TO_PREDICT = [
#     'b_DES_average',
#     'after_DES_total',
#     "b_DERS_total",
#     "after_DERS_total",
#     "diff_DERS",
#     "diff_DES"
# ]
#
# # 2. X Variables (Cluster Timepoints)
# X_CLUSTER_TIMEPOINTS = ['before', 't1', 't2', 't3', 'after']
#
# # 3. CV Settings
# N_SPLITS = 5
# RANDOM_STATE = 42
#
# df_for_analysis = None
#
# try:
#     # ===================================================================
#     #           PART 1: Data Preparation
#     # ===================================================================
#
#     # --- Load Cluster Data ---
#     print(f"Loading Cluster data...")
#     cluster_df = pd.read_csv(CLUSTER_DATA_WIDE_PATH)
#
#     # Ensure numeric (0/1)
#     for tp in X_CLUSTER_TIMEPOINTS:
#          cluster_df[tp] = pd.to_numeric(cluster_df[tp], errors='coerce')
#
#     # --- Load Y Data ---
#     print("Loading Y variables...")
#     columns_to_load = ['Subject_Code'] + Y_VARIABLES_TO_PREDICT
#     raw_df = pd.read_csv(raw_data_path, usecols=lambda c: c in columns_to_load)
#
#     # Ensure float
#     for y_col in Y_VARIABLES_TO_PREDICT:
#         if y_col in raw_df.columns:
#             raw_df[y_col] = raw_df[y_col].astype(np.float64, errors='ignore')
#
#     # --- Merge ---
#     df_merged = pd.merge(cluster_df, raw_df, on='Subject_Code', how='inner')
#
#     os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)
#     df_for_analysis = df_merged
#     print("--- Data Merged Successfully ---")
#
# except Exception as e:
#     print(f"Error loading data: {e}")
#     df_for_analysis = None
#
# # ===================================================================
# #           PART 2: LASSO Loop (Single Timepoint)
# # ===================================================================
#
# all_results_list = []
#
# if df_for_analysis is not None:
#
#     for y_col in Y_VARIABLES_TO_PREDICT:
#         print(f"\n=======================================================")
#         print(f"  Target (Y): {y_col}")
#         print(f"=======================================================")
#
#         # Loop through each timepoint separately
#         for tp in X_CLUSTER_TIMEPOINTS:
#
#             if tp not in df_for_analysis.columns:
#                 continue
#
#             # 1. Prepare Data
#             data_clean = df_for_analysis[[tp, y_col]].dropna()
#
#             if len(data_clean) < N_SPLITS * 2:
#                 continue
#
#             # X must be 2D for sklearn
#             X = data_clean[[tp]]
#             y = data_clean[y_col]
#
#             # Check if we actually have two clusters (0 and 1)
#             if X[tp].nunique() < 2:
#                 print(f"  Skipping {tp}: Only one cluster found.")
#                 continue
#
#             # --- 2. Scaling (Mandatory for Lasso) ---
#             scaler = StandardScaler()
#             X_scaled = scaler.fit_transform(X)
#
#             # --- 3. Train LassoCV Model ---
#             # This finds the best Alpha and fits the model
#             lasso_model = LassoCV(cv=N_SPLITS, random_state=RANDOM_STATE, max_iter=10000)
#             lasso_model.fit(X_scaled, y)
#
#             # Extract Coefs (Note: These are coefficients for the SCALED data)
#             best_alpha = lasso_model.alpha_
#             beta_scaled = lasso_model.coef_[0] # There is only one feature
#             intercept = lasso_model.intercept_
#
#             # --- 4. Cross Validation for R2 ---
#             kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
#             cv_scores = cross_validate(lasso_model, X_scaled, y, cv=kfold,
#                                      scoring={'r2': 'r2', 'neg_mse': 'neg_mean_squared_error'})
#
#             mean_r2 = np.mean(cv_scores['test_r2'])
#             mean_rmse = np.sqrt(-np.mean(cv_scores['test_neg_mse']))
#
#             print(f"  [{tp}] R²(CV): {mean_r2:.3f} | Beta(Scaled): {beta_scaled:.3f}")
#
#             # ===============================================================
#             #           Plot: Boxplot (If R2 > 0 and Beta != 0)
#             # ===============================================================
#             # We check abs(beta_scaled) > 0 because Lasso might zero it out
#             if mean_r2 > 0 and abs(beta_scaled) > 0.0001:
#
#                 plt.figure(figsize=(8, 6))
#
#                 # Boxplot of raw data (easier to interpret visually)
#                 sns.boxplot(x=tp, y=y_col, data=data_clean, palette="pastel")
#                 sns.stripplot(x=tp, y=y_col, data=data_clean, color='black', alpha=0.6, jitter=True)
#
#                 plt.title(f'Lasso Prediction: {y_col} by Cluster at {tp}')
#                 plt.xlabel(f'Cluster Assignment (0 vs 1) at {tp}')
#                 plt.ylabel(f'{y_col}')
#
#                 stats_text = (f'$R^2$ (CV) = {mean_r2:.3f}\n'
#                               f'Lasso Beta (Scaled) = {beta_scaled:.3f}\n'
#                               f'Alpha (Penalty) = {best_alpha:.4f}')
#
#                 plt.text(0.05, 0.95, stats_text, transform=plt.gca().transAxes,
#                          verticalalignment='top', fontsize=10,
#                          bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.8))
#
#                 plt.tight_layout()
#
#                 save_name = f"Lasso_Single_{y_col}_at_{tp}.png"
#                 plt.savefig(os.path.join(PLOTS_SAVE_DIR, save_name), dpi=100)
#                 plt.close()
#                 print(f"    ✅ Plot saved: {save_name}")
#             else:
#                 print(f"    ❌ No plot (R² <= 0 or Coef shrunk to 0)")
#
#             # Save results
#             all_results_list.append({
#                 'Y_Variable': y_col,
#                 'Timepoint': tp,
#                 'R2_CV': mean_r2,
#                 'RMSE_CV': mean_rmse,
#                 'Lasso_Beta_Scaled': beta_scaled,
#                 'Best_Penalty_Alpha': best_alpha,
#                 'N_samples': len(y)
#             })
#
#     # ===================================================================
#     #           PART 3: Save Summary
#     # ===================================================================
#     if all_results_list:
#         summary_df = pd.DataFrame(all_results_list)
#         summary_df = summary_df.sort_values(by=['Y_Variable', 'R2_CV'], ascending=[True, False])
#         summary_df.to_csv(SUMMARY_TABLE_SAVE_PATH, index=False)
#         print(f"\n✅ Summary saved to: {SUMMARY_TABLE_SAVE_PATH}")
#     else:
#         print("No results generated.")

Loading Cluster data...
Error loading data: [Errno 2] No such file or directory: 'data/only_Q_outputs/combined/timepoints_file_inverted_2.csv'
